## Step 60: Generate SWC PREC Datasets

Purpose: combine SWC stations with CREAT3 projected data, historic extreme event data and rainfall 
distribution datasets.  Partition data by station into individual feature classes modelling the 
final text file outputs.


In [1]:
import arcpy;
import os,sys;
import datetime,importlib;

print("Step 60: Generate SWC PREC Datasets");

import swc_resources;
importlib.reload(swc_resources);
rez = swc_resources.rez();

near_max_historic_val  = 1;
near_max_historic_unit = 'degree';
near_max_rainfall_val  = 1;
near_max_rainfall_unit = 'degree';

stations       = rez['results'] + os.sep + 'SWC_Station_Universe';
stations_cnt   = arcpy.GetCount_management(stations)[0];

gridinterp     = rez['results'] + os.sep + 'CRWU_CREAT_Grid_Projections_DATA_INTERP';
gridinterp_cnt = arcpy.GetCount_management(gridinterp)[0];

hist      = rez['source'] + os.sep + 'CRWU_CREAT_Historic_Climate_Stations';
hist_cnt  = arcpy.GetCount_management(hist)[0];

rfcov     = rez['results'] + os.sep + 'RainfallDistribution';
rfcov_cnt = arcpy.GetCount_management(rfcov)[0];

print("  Station Count          : " + str(stations_cnt));
print("  Interpolated Grid Count: " + str(gridinterp_cnt));
print("  Historic Station Count : " + str(hist_cnt));
print("  Rainfall Distribution Coverage Count: " + str(rfcov_cnt));


Step 60: Generate SWC PREC Datasets
  Station Count          : 5144
  Interpolated Grid Count: 6332
  Historic Station Count : 11165
  Rainfall Distribution Coverage Count: 19614


 ### 60.010: Calculate nearest historical station

In [2]:
%%time

input60010  = rez['results'] + os.sep + 'SWC_Station_Universe';
output60010 = rez['working']  + os.sep + 'D4EMLite_PREC_Details_WRK2';

if arcpy.Exists(output60010):
    arcpy.Delete_management(output60010);
   
print("  copying to new work stations");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = input60010
    ,out_path    = os.path.dirname(output60010)
    ,out_name    = os.path.basename(output60010)
);

print("  add tracking fields for joined data");
arcpy.management.AddFields(
     in_table = output60010
    ,field_description = [
         ['has_creat3_grid_data'     ,'TEXT','Has CREAT3 Grid Data', 1]
        ,['is_interp_grid_data'      ,'TEXT','Is Interpolated Grid Data', 1]
        ,['has_creat3_histstorm_data','TEXT','Has CREAT3 Historic Storm Data',1]
        ,['has_rainfall_data'        ,'TEXT','Has Rainfall Data',1]
    ]
);

print("  calculating the nearest historic station");
arcpy.analysis.Near(
     in_features   = output60010
    ,near_features = hist
    ,location      = 'NO_LOCATION'
    ,method        = 'PLANAR'
);

print("  adding indexes");
arcpy.management.AddIndex(
     in_table        = output60010
    ,fields          = 'NEAR_FID'
    ,index_name      = 'NEAR_FID_IDX'
);

print("  joining in historic storms information");
flds = [
     'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
];

arcpy.management.JoinField(
     in_data         = output60010
    ,in_field        = 'NEAR_FID'
    ,join_table      = hist
    ,join_field      = 'OBJECTID'
    ,fields          = flds
);

flds = [
     'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
];

for fld in flds:

    arcpy.management.CalculateField(
         in_table        = output60010
        ,field           = fld
        ,expression      = "nIfNone(!" + fld + "!,!NEAR_DIST!)"
        ,expression_type = 'PYTHON3'
        ,code_block      = """
def nIfNone(pval,pdist):
    if pdist > """ + str(near_max_historic_val) + """ or pdist == -1:
        return 0;
    else:
        return pval;
"""
    );

print("  tracking unmatched stations");
arcpy.management.CalculateField(
     in_table        = output60010
    ,field           = 'has_creat3_histstorm_data'
    ,expression      = "nIfNone(!IntensePrecip_Historic_5Year_24!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = """
def nIfNone(pin):
    if pin is None or pin <= 0:
       return 'N';
    else:
       return 'Y';
"""
);

cnt = arcpy.GetCount_management(output60010)[0];
print("  Station Count: " + str(cnt));


  copying to new work stations
  add tracking fields for joined data
  calculating the nearest historic station
  adding indexes
  joining in historic storms information
  tracking unmatched stations
  Station Count: 5144
Wall time: 15.7 s


### 60.020: Calculate the containing CREAT3 grid with temperture and precipitation data for stations

In [3]:
%%time

flds = [
     'TEMP_2035_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2035_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2035_HOTDRY_APR_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2035_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2035_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2035_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2035_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2035_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2035_HOTDRY_ANNUAL_CHANGEF'
    
    ,'TEMP_2035_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2035_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2035_CENTRAL_APR_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2035_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2035_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2035_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2035_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2035_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2035_CENTRAL_ANNUAL_CHANGE'
    
    ,'TEMP_2035_WETWARM_JAN_CHANGEF'
    ,'TEMP_2035_WETWARM_FEB_CHANGEF'
    ,'TEMP_2035_WETWARM_MAR_CHANGEF'
    ,'TEMP_2035_WETWARM_APR_CHANGEF'
    ,'TEMP_2035_WETWARM_MAY_CHANGEF'
    ,'TEMP_2035_WETWARM_JUN_CHANGEF'
    ,'TEMP_2035_WETWARM_JUL_CHANGEF'
    ,'TEMP_2035_WETWARM_AUG_CHANGEF'
    ,'TEMP_2035_WETWARM_SEP_CHANGEF'
    ,'TEMP_2035_WETWARM_OCT_CHANGEF'
    ,'TEMP_2035_WETWARM_NOV_CHANGEF'
    ,'TEMP_2035_WETWARM_DEC_CHANGEF'
    ,'TEMP_2035_WETWARM_ANNUAL_CHANGE'
    
    ,'TEMP_2060_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2060_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2060_HOTDRY_APR_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2060_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2060_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2060_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2060_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2060_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2060_HOTDRY_ANNUAL_CHANGEF'
    
    ,'TEMP_2060_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2060_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2060_CENTRAL_APR_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2060_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2060_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2060_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2060_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2060_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2060_CENTRAL_ANNUAL_CHANGE'
    
    ,'TEMP_2060_WETWARM_JAN_CHANGEF'
    ,'TEMP_2060_WETWARM_FEB_CHANGEF'
    ,'TEMP_2060_WETWARM_MAR_CHANGEF'
    ,'TEMP_2060_WETWARM_APR_CHANGEF'
    ,'TEMP_2060_WETWARM_MAY_CHANGEF'
    ,'TEMP_2060_WETWARM_JUN_CHANGEF'
    ,'TEMP_2060_WETWARM_JUL_CHANGEF'
    ,'TEMP_2060_WETWARM_AUG_CHANGEF'
    ,'TEMP_2060_WETWARM_SEP_CHANGEF'
    ,'TEMP_2060_WETWARM_OCT_CHANGEF'
    ,'TEMP_2060_WETWARM_NOV_CHANGEF'
    ,'TEMP_2060_WETWARM_DEC_CHANGEF'
    ,'TEMP_2060_WETWARM_ANNUAL_CHANGE'
    
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'

    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'

    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
];

wrkstat2 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK2';
wrkstat3 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK3';

if arcpy.Exists(wrkstat3):
    arcpy.Delete_management(wrkstat3);
    
print("  calculating the containing CREAT3 grid with precipitation data");
arcpy.analysis.SpatialJoin(
     target_features   = wrkstat2
    ,join_features     = gridinterp
    ,out_feature_class = wrkstat3
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'WITHIN'
);

print("  updating tracking fields for grid source");
arcpy.management.CalculateField(
     in_table        = wrkstat3
    ,field           = 'has_creat3_grid_data'
    ,expression      = "nIfNone(!PRECIP_2035_HOTDRY_JAN_CHANGEPC!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = """
def nIfNone(pin):
    if pin is None:
       return 'N';
    else:
       return 'Y';
"""
);

arcpy.management.CalculateField(
     in_table        = wrkstat3
    ,field           = 'is_interp_grid_data'
    ,expression      = "!is_interpolated!"
    ,expression_type = 'PYTHON3'
);

print("  cleaning up unneeded fields")
arcpy.management.DeleteField(
     in_table   = wrkstat3
    ,drop_field = [
         'join_count'
        ,'TARGET_FID'
        ,'CREAT_ID'
        ,'GRIDCODE'
        ,'HurrGridID'
        ,'grid_over_land'
        ,'Longitude_Centroid'
        ,'Latitude_Centroid'
        ,'Longitude_Min'
        ,'Latitude_Min'
        ,'Longitude_Max'
        ,'Latitude_Max'
        ,'is_interpolated'
    ]
);

print("  zeroing out unmatched stations");
for fld in flds:
   
    arcpy.management.CalculateField(
         in_table        = wrkstat3
        ,field           = fld
        ,expression      = "zeroIfNone(!" + fld + "!)"
        ,expression_type = 'PYTHON3'
        ,code_block      = """
def zeroIfNone(pin):
    if pin is None:
       return 0;
    else:
       return pin;
"""
    );
    
cnt = arcpy.GetCount_management(wrkstat3)[0];
print("  Station Count: " + str(cnt));


  calculating the containing CREAT3 grid with precipitation data
  updating tracking fields for grid source
  cleaning up unneeded fields
  zeroing out unmatched stations
  Station Count: 5144
Wall time: 3min 15s


### 60.030: Determine Precipitation RF value for stations

In [4]:
%%time

wrkstat3 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK3';
wrkstat4 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK4';

if arcpy.Exists(wrkstat4):
    arcpy.Delete_management(wrkstat4);
    
print("  copying to new work stations");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = wrkstat3
    ,out_path    = os.path.dirname(wrkstat4)
    ,out_name    = os.path.basename(wrkstat4)
);

print("  calculating the nearest rainfall coverage polygon ",end='');
print("within " + str(near_max_rainfall_val) + " " + near_max_rainfall_unit + "(s)");
arcpy.analysis.Near(
     in_features   = wrkstat4
    ,near_features = rfcov
    ,location      = 'NO_LOCATION'
    ,method        = 'PLANAR'
);

print("  joining in rainfall distribution information");
arcpy.management.JoinField(
     in_data         = wrkstat4
    ,in_field        = 'NEAR_FID'
    ,join_table      = rfcov
    ,join_field      = 'OBJECTID'
    ,fields          = ['rf_value']
);

arcpy.management.CalculateField(
     in_table        = wrkstat4
    ,field           = 'rf_value'
    ,expression      = "nIfNone(!rf_value!,!NEAR_DIST!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = """
def nIfNone(pval,pdist):
    if pdist > """ + str(near_max_rainfall_val) + """ or pdist == -1:
        return None;
    else:
        return pval;
"""
);

print("  tracking unmatched stations");
z = arcpy.management.CalculateField(
     in_table        = wrkstat4
    ,field           = 'has_rainfall_data'
    ,expression      = "nIfNone(!rf_value!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = """
def nIfNone(pin):
    if pin is None:
       return 'N';
    else:
       return 'Y';
"""
);

print("  cleaning up unneeded fields")
z = arcpy.management.DeleteField(
     in_table   = wrkstat4
    ,drop_field = [
         'NEAR_FID'
        ,'NEAR_DIST'
    ]
);

cnt = arcpy.GetCount_management(wrkstat4)[0];
print("  Station Count: " + str(cnt));


  copying to new work stations
  calculating the nearest rainfall coverage polygon within 1 degree(s)
  joining in rainfall distribution information
  tracking unmatched stations
  cleaning up unneeded fields
  Station Count: 5144
Wall time: 11min 34s


### TEMP and PRECIP Table Makers

Functions to create empty TEMP and PRECIP tables

In [5]:
def createTEMP(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createPREC(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createGEVdepth(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId'   ,'TEXT'  ,'StationId',14]
            ,['z5yrann_mm'  ,'DOUBLE','5yrann_mm']
            ,['z10yrann_mm' ,'DOUBLE','10yrann_mm']
            ,['z15yrann_mm' ,'DOUBLE','15yrann_mm']
            ,['z30yrann_mm' ,'DOUBLE','30yrann_mm']
            ,['z50yrann_mm' ,'DOUBLE','50yrann_mm']
            ,['z100yrann_mm','DOUBLE','100yrann_mm']
         ]
    );
    

### 60.040: Create fresh tables for TEMP and PREC extracts 

In [6]:
%%time

print("  creating fresh TEMP feature classes");
createTEMP(rez['results'],'TEMP2035HotDry');

createTEMP(rez['results'],'TEMP2035Central');

createTEMP(rez['results'],'TEMP2035WetWarm');

createTEMP(rez['results'],'TEMP2060HotDry');

createTEMP(rez['results'],'TEMP2060Central');

createTEMP(rez['results'],'TEMP2060WetWarm');

print("  creating fresh PREC feature classes");
createPREC(rez['results'],'PREC2035HotDry');

createPREC(rez['results'],'PREC2035Central');

createPREC(rez['results'],'PREC2035WetWarm');

createPREC(rez['results'],'PREC2060HotDry');

createPREC(rez['results'],'PREC2060Central');

createPREC(rez['results'],'PREC2060WetWarm');

print("  creating fresh GEVddepth feature classes");
createGEVdepth(rez['results'],'GEVdepth2035Stormy');

createGEVdepth(rez['results'],'GEVdepth2035LessStormy');

createGEVdepth(rez['results'],'GEVdepth2060Stormy');

createGEVdepth(rez['results'],'GEVdepth2060LessStormy');

createGEVdepth(rez['results'],'GEVdepth_historical');

print("  creating fresh PREC_SCS_Types feature class");
if arcpy.Exists(rez['results'] + os.sep + 'PREC_SCS_Types'):
    arcpy.Delete_management(rez['results'] + os.sep + 'PREC_SCS_Types');
        
arcpy.CreateFeatureclass_management(
     out_path      = rez['results']
    ,out_name      = 'PREC_SCS_Types'
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

z = arcpy.management.AddFields(
     in_table = rez['results'] + os.sep + 'PREC_SCS_Types'
    ,field_description = [
         ['StationId'                 ,'TEXT','StationId',14]
        ,['Rainfall_Distribution_Type','TEXT','Rainfall_Distribution_Type',255]
     ]
);


  creating fresh TEMP feature classes
  creating fresh PREC feature classes
  creating fresh GEVddepth feature classes
  creating fresh PREC_SCS_Types feature class
Wall time: 48.8 s


### 60.050: Write out datasets to individual feature classes

In [7]:
%%time

wrkstat4 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK4';

fldin = [
     'StationId'
    # 1
    ,'TEMP_2035_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2035_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2035_HOTDRY_APR_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2035_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2035_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2035_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2035_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2035_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2035_HOTDRY_ANNUAL_CHANGEF'
    # 14
    ,'TEMP_2035_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2035_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2035_CENTRAL_APR_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2035_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2035_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2035_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2035_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2035_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2035_CENTRAL_ANNUAL_CHANGE'
    # 27
    ,'TEMP_2035_WETWARM_JAN_CHANGEF'
    ,'TEMP_2035_WETWARM_FEB_CHANGEF'
    ,'TEMP_2035_WETWARM_MAR_CHANGEF'
    ,'TEMP_2035_WETWARM_APR_CHANGEF'
    ,'TEMP_2035_WETWARM_MAY_CHANGEF'
    ,'TEMP_2035_WETWARM_JUN_CHANGEF'
    ,'TEMP_2035_WETWARM_JUL_CHANGEF'
    ,'TEMP_2035_WETWARM_AUG_CHANGEF'
    ,'TEMP_2035_WETWARM_SEP_CHANGEF'
    ,'TEMP_2035_WETWARM_OCT_CHANGEF'
    ,'TEMP_2035_WETWARM_NOV_CHANGEF'
    ,'TEMP_2035_WETWARM_DEC_CHANGEF'
    ,'TEMP_2035_WETWARM_ANNUAL_CHANGE'
    # 40
    ,'TEMP_2060_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2060_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2060_HOTDRY_APR_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2060_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2060_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2060_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2060_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2060_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2060_HOTDRY_ANNUAL_CHANGEF'
    # 53
    ,'TEMP_2060_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2060_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2060_CENTRAL_APR_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2060_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2060_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2060_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2060_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2060_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2060_CENTRAL_ANNUAL_CHANGE'
    # 66
    ,'TEMP_2060_WETWARM_JAN_CHANGEF'
    ,'TEMP_2060_WETWARM_FEB_CHANGEF'
    ,'TEMP_2060_WETWARM_MAR_CHANGEF'
    ,'TEMP_2060_WETWARM_APR_CHANGEF'
    ,'TEMP_2060_WETWARM_MAY_CHANGEF'
    ,'TEMP_2060_WETWARM_JUN_CHANGEF'
    ,'TEMP_2060_WETWARM_JUL_CHANGEF'
    ,'TEMP_2060_WETWARM_AUG_CHANGEF'
    ,'TEMP_2060_WETWARM_SEP_CHANGEF'
    ,'TEMP_2060_WETWARM_OCT_CHANGEF'
    ,'TEMP_2060_WETWARM_NOV_CHANGEF'
    ,'TEMP_2060_WETWARM_DEC_CHANGEF'
    ,'TEMP_2060_WETWARM_ANNUAL_CHANGE'
    # 79 
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'
    # 92
    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'
    # 105
    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'
    # 118
    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'
    # 131
    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'
    # 144
    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'
    # 157
    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    # 163
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    # 169
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    # 175
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
    # 181
    ,'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
    # 187
    ,'rf_value'
];

fldout = [
     'StationId'
    ,'Jan'
    ,'Feb'
    ,'Mar'
    ,'Apr'
    ,'May'
    ,'Jun'
    ,'Jul'
    ,'Aug'
    ,'Sep'
    ,'Oct'
    ,'Nov'
    ,'Dec'
    ,'Ann'
];

fldgev = [
     'StationId'
    ,'z5yrann_mm'
    ,'z10yrann_mm'
    ,'z15yrann_mm'
    ,'z30yrann_mm'
    ,'z50yrann_mm'
    ,'z100yrann_mm'
];

fldscs = [
     'StationId'
    ,'Rainfall_Distribution_Type'
];

print("  truncate datasets for safety");
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2035HotDry');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2035Central');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2035WetWarm');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2060HotDry');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2060Central');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'TEMP2060WetWarm');

arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2035HotDry');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2035Central');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2035WetWarm');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2060HotDry');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2060Central');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC2060WetWarm');

arcpy.TruncateTable_management(rez['results'] + os.sep + 'GEVdepth2035Stormy');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'GEVdepth2035LessStormy');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'GEVdepth2060Stormy');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'GEVdepth2060LessStormy');
arcpy.TruncateTable_management(rez['results'] + os.sep + 'GEVdepth_historical');

arcpy.TruncateTable_management(rez['results'] + os.sep + 'PREC_SCS_Types');

print("  opening an editor session to support multiple cursors");
with arcpy.da.Editor(rez['results']) as edit:
      
    curs_TEMP2035HotDry = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2035HotDry'   
        ,field_names = fldout
    );
    
    curs_TEMP2035Central = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2035Central'   
        ,field_names = fldout
    );
        
    curs_TEMP2035WetWarm = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2035WetWarm'   
        ,field_names = fldout
    );
    
    curs_TEMP2060HotDry = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2060HotDry'   
        ,field_names = fldout
    );
    
    curs_TEMP2060Central = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2060Central'   
        ,field_names = fldout
    );
        
    curs_TEMP2060WetWarm = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'TEMP2060WetWarm'   
        ,field_names = fldout
    );
    
    curs_PREC2035HotDry = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2035HotDry'   
        ,field_names = fldout
    );
    
    curs_PREC2035Central = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2035Central'   
        ,field_names = fldout
    );
        
    curs_PREC2035WetWarm = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2035WetWarm'   
        ,field_names = fldout
    );
    
    curs_PREC2060HotDry = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2060HotDry'   
        ,field_names = fldout
    );
    
    curs_PREC2060Central = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2060Central'   
        ,field_names = fldout
    );
        
    curs_PREC2060WetWarm = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC2060WetWarm'   
        ,field_names = fldout
    );
    
    curs_GEVdepth2035Stormy = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'GEVdepth2035Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2035LessStormy = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'GEVdepth2035LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepth2060Stormy = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'GEVdepth2060Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2060LessStormy = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'GEVdepth2060LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepth_historical = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'GEVdepth_historical'   
        ,field_names = fldgev
    );
    
    curs_PREC_SCS_Types = arcpy.da.InsertCursor(
         in_table    = rez['results'] + os.sep + 'PREC_SCS_Types'   
        ,field_names = fldscs
    );
    
    print("  writing the station data to individual feature classes");
    with arcpy.da.SearchCursor(
         in_table    = wrkstat4
        ,field_names = fldin
    ) as incur:
    
        for row in incur:
            
            # TEMP 2035 HOTDRY
            curs_TEMP2035HotDry.insertRow((
                 row[0]
                ,round(row[1]  ,2)
                ,round(row[2]  ,2)
                ,round(row[3]  ,2)
                ,round(row[4]  ,2)
                ,round(row[5]  ,2)
                ,round(row[6]  ,2)
                ,round(row[7]  ,2)
                ,round(row[8]  ,2)
                ,round(row[9]  ,2)
                ,round(row[10] ,2)
                ,round(row[11] ,2)
                ,round(row[12] ,2)
                ,round(row[13] ,2)
            ));
            
            # TEMP 2035 CENTRAL
            curs_TEMP2035Central.insertRow((
                 row[0]
                ,round(row[14] ,2)
                ,round(row[15] ,2)
                ,round(row[16] ,2)
                ,round(row[17] ,2)
                ,round(row[18] ,2)
                ,round(row[19] ,2)
                ,round(row[20] ,2)
                ,round(row[21] ,2)
                ,round(row[22] ,2)
                ,round(row[23] ,2)
                ,round(row[24] ,2)
                ,round(row[25] ,2)
                ,round(row[26] ,2)
            ));
            
            # TEMP 2035 WETWARM
            curs_TEMP2035WetWarm.insertRow((
                 row[0]
                ,round(row[27] ,2)
                ,round(row[28] ,2)
                ,round(row[29] ,2)
                ,round(row[30] ,2)
                ,round(row[31] ,2)
                ,round(row[32] ,2)
                ,round(row[33] ,2)
                ,round(row[34] ,2)
                ,round(row[35] ,2)
                ,round(row[36] ,2)
                ,round(row[37] ,2)
                ,round(row[38] ,2)
                ,round(row[39] ,2)
            ));
            
            # TEMP 2060 HOTDRY
            curs_TEMP2060HotDry.insertRow((
                 row[0]
                ,round(row[40] ,2)
                ,round(row[41] ,2)
                ,round(row[42] ,2)
                ,round(row[43] ,2)
                ,round(row[44] ,2)
                ,round(row[45] ,2)
                ,round(row[46] ,2)
                ,round(row[47] ,2)
                ,round(row[48] ,2)
                ,round(row[49] ,2)
                ,round(row[50] ,2)
                ,round(row[51] ,2)
                ,round(row[52] ,2)
            ));
            
            # TEMP 2060 CENTRAL
            curs_TEMP2060Central.insertRow((
                 row[0]
                ,round(row[53] ,2)
                ,round(row[54] ,2)
                ,round(row[55] ,2)
                ,round(row[56] ,2)
                ,round(row[57] ,2)
                ,round(row[58] ,2)
                ,round(row[59] ,2)
                ,round(row[60] ,2)
                ,round(row[61] ,2)
                ,round(row[62] ,2)
                ,round(row[63] ,2)
                ,round(row[64] ,2)
                ,round(row[65] ,2)
            ));
            
            # TEMP 2060 WETWARM
            curs_TEMP2060WetWarm.insertRow((
                 row[0]
                ,round(row[66] ,2)
                ,round(row[67] ,2)
                ,round(row[68] ,2)
                ,round(row[69] ,2)
                ,round(row[70] ,2)
                ,round(row[71] ,2)
                ,round(row[72] ,2)
                ,round(row[73] ,2)
                ,round(row[74] ,2)
                ,round(row[75] ,2)
                ,round(row[76] ,2)
                ,round(row[77] ,2)
                ,round(row[78] ,2)
            ));
            
            # PRECIP 2035 HOTDRY
            curs_PREC2035HotDry.insertRow((
                 row[0]
                ,round(row[79] ,2)
                ,round(row[80] ,2)
                ,round(row[81] ,2)
                ,round(row[82] ,2)
                ,round(row[83] ,2)
                ,round(row[84] ,2)
                ,round(row[85] ,2)
                ,round(row[86] ,2)
                ,round(row[87] ,2)
                ,round(row[88] ,2)
                ,round(row[89] ,2)
                ,round(row[90] ,2)
                ,round(row[91] ,2)
            ));
            
            # PRECIP 2035 CENTRAL
            curs_PREC2035Central.insertRow((
                 row[0]
                ,round(row[92] ,2)
                ,round(row[93] ,2)
                ,round(row[94] ,2)
                ,round(row[95] ,2)
                ,round(row[96] ,2)
                ,round(row[97] ,2)
                ,round(row[98] ,2)
                ,round(row[99] ,2)
                ,round(row[100],2)
                ,round(row[101],2)
                ,round(row[102],2)
                ,round(row[103],2)
                ,round(row[104],2)
            ));
            
            # PRECIP 2035 WETWARM
            curs_PREC2035WetWarm.insertRow((
                 row[0]
                ,round(row[105],2)
                ,round(row[106],2)
                ,round(row[107],2)
                ,round(row[108],2)
                ,round(row[109],2)
                ,round(row[110],2)
                ,round(row[111],2)
                ,round(row[112],2)
                ,round(row[113],2)
                ,round(row[114],2)
                ,round(row[115],2)
                ,round(row[116],2)
                ,round(row[117],2)
            ));
            
            # PRECIP 2060 HOTDRY
            curs_PREC2060HotDry.insertRow((
                 row[0]
                ,round(row[118],2)
                ,round(row[119],2)
                ,round(row[120],2)
                ,round(row[121],2)
                ,round(row[122],2)
                ,round(row[123],2)
                ,round(row[124],2)
                ,round(row[125],2)
                ,round(row[126],2)
                ,round(row[127],2)
                ,round(row[128],2)
                ,round(row[129],2)
                ,round(row[130],2)
            ));
            
            # PRECIP 2060 CENTRAL
            curs_PREC2060Central.insertRow((
                 row[0]
                ,round(row[131],2)
                ,round(row[132],2)
                ,round(row[133],2)
                ,round(row[134],2)
                ,round(row[135],2)
                ,round(row[136],2)
                ,round(row[137],2)
                ,round(row[138],2)
                ,round(row[139],2)
                ,round(row[140],2)
                ,round(row[141],2)
                ,round(row[142],2)
                ,round(row[143],2)
            ));
            
            # PRECIP 2060 WETWARM
            curs_PREC2060WetWarm.insertRow((
                 row[0]
                ,round(row[144],2)
                ,round(row[145],2)
                ,round(row[146],2)
                ,round(row[147],2)
                ,round(row[148],2)
                ,round(row[149],2)
                ,round(row[150],2)
                ,round(row[151],2)
                ,round(row[152],2)
                ,round(row[153],2)
                ,round(row[154],2)
                ,round(row[155],2)
                ,round(row[156],2)
            ));
            
            IntensePrecip_Historic_5Year_mm   = row[181] * 25.4;
            IntensePrecip_Historic_10Year_mm  = row[182] * 25.4;
            IntensePrecip_Historic_15Year_mm  = row[183] * 25.4;
            IntensePrecip_Historic_30Year_mm  = row[184] * 25.4;
            IntensePrecip_Historic_50Year_mm  = row[185] * 25.4;
            IntensePrecip_Historic_100Year_mm = row[186] * 25.4;
            
            # STORMY 2035
            curs_GEVdepth2035Stormy.insertRow((
                 row[0]
                ,round(IntensePrecip_Historic_5Year_mm   * (1 + row[157] / 100),2)
                ,round(IntensePrecip_Historic_10Year_mm  * (1 + row[158] / 100),2)
                ,round(IntensePrecip_Historic_15Year_mm  * (1 + row[159] / 100),2)
                ,round(IntensePrecip_Historic_30Year_mm  * (1 + row[160] / 100),2)
                ,round(IntensePrecip_Historic_50Year_mm  * (1 + row[161] / 100),2)
                ,round(IntensePrecip_Historic_100Year_mm * (1 + row[162] / 100),2)
            ));
            
            # STORMY 2060
            curs_GEVdepth2060Stormy.insertRow((
                 row[0]
                ,round(IntensePrecip_Historic_5Year_mm   * (1 + row[163] / 100),2)
                ,round(IntensePrecip_Historic_10Year_mm  * (1 + row[164] / 100),2)
                ,round(IntensePrecip_Historic_15Year_mm  * (1 + row[165] / 100),2)
                ,round(IntensePrecip_Historic_30Year_mm  * (1 + row[166] / 100),2)
                ,round(IntensePrecip_Historic_50Year_mm  * (1 + row[167] / 100),2)
                ,round(IntensePrecip_Historic_100Year_mm * (1 + row[168] / 100),2)
            ));
            
            # LESS STORMY 2035
            curs_GEVdepth2035LessStormy.insertRow((
                 row[0]
                ,round(IntensePrecip_Historic_5Year_mm   * (1 + row[169] / 100),2)
                ,round(IntensePrecip_Historic_10Year_mm  * (1 + row[170] / 100),2)
                ,round(IntensePrecip_Historic_15Year_mm  * (1 + row[171] / 100),2)
                ,round(IntensePrecip_Historic_30Year_mm  * (1 + row[172] / 100),2)
                ,round(IntensePrecip_Historic_50Year_mm  * (1 + row[173] / 100),2)
                ,round(IntensePrecip_Historic_100Year_mm * (1 + row[174] / 100),2)
            ));
            
            # LESS STORMY 2060
            curs_GEVdepth2060LessStormy.insertRow((
                 row[0]
                ,round(IntensePrecip_Historic_5Year_mm   * (1 + row[175] / 100),2)
                ,round(IntensePrecip_Historic_10Year_mm  * (1 + row[176] / 100),2)
                ,round(IntensePrecip_Historic_15Year_mm  * (1 + row[177] / 100),2)
                ,round(IntensePrecip_Historic_30Year_mm  * (1 + row[178] / 100),2)
                ,round(IntensePrecip_Historic_50Year_mm  * (1 + row[179] / 100),2)
                ,round(IntensePrecip_Historic_100Year_mm * (1 + row[180] / 100),2)
            ));
            
            # Intense Precipitation Historic in mm
            curs_GEVdepth_historical.insertRow((
                 row[0]
                ,round(IntensePrecip_Historic_5Year_mm  ,2)
                ,round(IntensePrecip_Historic_10Year_mm ,2)
                ,round(IntensePrecip_Historic_15Year_mm ,2)
                ,round(IntensePrecip_Historic_30Year_mm ,2)
                ,round(IntensePrecip_Historic_50Year_mm ,2)
                ,round(IntensePrecip_Historic_100Year_mm,2)
            ));
            
            curs_PREC_SCS_Types.insertRow((
                 row[0]
                ,row[187]
            ));


  truncate datasets for safety
  opening an editor session to support multiple cursors
  writing the station data to individual feature classes
Wall time: 19.7 s


### 60.060: Finalize Results and cleanup unneeded fields

In [8]:
%%time

wrkstat4 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK4';

rezstat = rez['results'] + os.sep + 'D4EMLite_PREC_Details';

if arcpy.Exists(rezstat):
    arcpy.Delete_management(rezstat);
   
print("  copying to workstations results");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = wrkstat4
    ,out_path    = os.path.dirname(rezstat)
    ,out_name    = os.path.basename(rezstat)
);

print("  cleaning up the fieldset");
flds_to_keep = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'has_creat3_grid_data'
    ,'is_interp_grid_data'
    ,'has_creat3_histstorm_data'
    ,'has_rainfall_data'
    ,'Shape'
    ,'OBJECTID'
];

flds = [f.name for f in arcpy.ListFields(rezstat)]
delete_flds = list(set(flds) - set(flds_to_keep));

z = arcpy.management.DeleteField(
     in_table   = rezstat
    ,drop_field = delete_flds
);


  copying to workstations results
  cleaning up the fieldset
Wall time: 1min


### 60.070: Review and QA

QA Products:

1. comparison of interpolated values to nearest values saved to **step60qa.txt**


In [9]:
%%time

wrkstat4 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK4';

qa_flds = [
     # 0
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    # 10
    ,'Value'
    ,'StaNam'
    ,'has_creat3_grid_data'
    ,'is_interp_grid_data'
    ,'has_creat3_histstorm_data'
    ,'has_rainfall_data'
    ,'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    # 20
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
    ,'TEMP_2035_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2035_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2035_HOTDRY_APR_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2035_HOTDRY_AUG_CHANGEF'
    # 30
    ,'TEMP_2035_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2035_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2035_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2035_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2035_HOTDRY_ANNUAL_CHANGEF'
    ,'TEMP_2035_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2035_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2035_CENTRAL_APR_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAY_CHANGEF'
    # 40
    ,'TEMP_2035_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2035_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2035_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2035_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2035_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2035_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2035_CENTRAL_ANNUAL_CHANGE'
    ,'TEMP_2035_WETWARM_JAN_CHANGEF'
    ,'TEMP_2035_WETWARM_FEB_CHANGEF'
    # 50
    ,'TEMP_2035_WETWARM_MAR_CHANGEF'
    ,'TEMP_2035_WETWARM_APR_CHANGEF'
    ,'TEMP_2035_WETWARM_MAY_CHANGEF'
    ,'TEMP_2035_WETWARM_JUN_CHANGEF'
    ,'TEMP_2035_WETWARM_JUL_CHANGEF'
    ,'TEMP_2035_WETWARM_AUG_CHANGEF'
    ,'TEMP_2035_WETWARM_SEP_CHANGEF'
    ,'TEMP_2035_WETWARM_OCT_CHANGEF'
    ,'TEMP_2035_WETWARM_NOV_CHANGEF'
    ,'TEMP_2035_WETWARM_DEC_CHANGEF'
    # 60
    ,'TEMP_2035_WETWARM_ANNUAL_CHANGE'
    ,'TEMP_2060_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2060_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2060_HOTDRY_APR_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2060_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2060_HOTDRY_SEP_CHANGEF'
    # 70
    ,'TEMP_2060_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2060_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2060_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2060_HOTDRY_ANNUAL_CHANGEF'
    ,'TEMP_2060_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2060_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2060_CENTRAL_APR_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUN_CHANGEF'
    # 80
    ,'TEMP_2060_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2060_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2060_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2060_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2060_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2060_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2060_CENTRAL_ANNUAL_CHANGE'
    ,'TEMP_2060_WETWARM_JAN_CHANGEF'
    ,'TEMP_2060_WETWARM_FEB_CHANGEF'
    ,'TEMP_2060_WETWARM_MAR_CHANGEF'
    # 90
    ,'TEMP_2060_WETWARM_APR_CHANGEF'
    ,'TEMP_2060_WETWARM_MAY_CHANGEF'
    ,'TEMP_2060_WETWARM_JUN_CHANGEF'
    ,'TEMP_2060_WETWARM_JUL_CHANGEF'
    ,'TEMP_2060_WETWARM_AUG_CHANGEF'
    ,'TEMP_2060_WETWARM_SEP_CHANGEF'
    ,'TEMP_2060_WETWARM_OCT_CHANGEF'
    ,'TEMP_2060_WETWARM_NOV_CHANGEF'
    ,'TEMP_2060_WETWARM_DEC_CHANGEF'
    ,'TEMP_2060_WETWARM_ANNUAL_CHANGE' 
    # 100
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    # 110
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'
    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    # 120
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'
    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    # 130
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'
    # 150
    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    # 160
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'
    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    # 170
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'
    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    # 180
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'
    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    # 190
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    # 200
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    # 210
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
    ,'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
    # 219
    ,'rf_value'
]

print("  writing out QA report");
with open(rez['qa'] + os.sep + 'step60qa.txt',"w",encoding="utf-8") as out:
    out.write("Step 60 QA Station Review\n");
    out.write(datetime.datetime.now().isoformat() + "\n");
    
    val = "";
    for fld in qa_flds:
        val += '"' + fld + '",';
    out.write(val.rstrip(',') + "\n");

    with arcpy.da.SearchCursor(
         in_table    = wrkstat4
        ,field_names = qa_flds
        ,sql_clause  = (None,'ORDER BY StationId')
    ) as cur:
        
        for row in cur:
            
            val = "";
            for item in row:
                if item is None:
                    val += ",";
                elif type(item) == int or type(item) == float:
                    val += str(item) + ",";
                else:
                    val += '"' + item + '",';
                    
            out.write(val.rstrip(',') + "\n");

D4EMLite_PREC_Details_cnt  = arcpy.GetCount_management(rez['results'] + os.sep + 'D4EMLite_PREC_Details')[0];

TEMP2035HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2035HotDry')[0];
TEMP2035Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2035Central')[0];
TEMP2035WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2035WetWarm')[0];

TEMP2060HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2060HotDry')[0];
TEMP2060Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2060Central')[0];
TEMP2060WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'TEMP2060WetWarm')[0];

PREC2035HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035HotDry')[0];
PREC2035Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035Central')[0];
PREC2035WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035WetWarm')[0];

PREC2060HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060HotDry')[0];
PREC2060Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060Central')[0];
PREC2060WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060WetWarm')[0];

GEVdepth2035Stormy_cnt     = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2035Stormy')[0];
GEVdepth2035LessStormy_cnt = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2035LessStormy')[0];

GEVdepth2060Stormy_cnt     = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2060Stormy')[0];
GEVdepth2060LessStormy_cnt = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2060LessStormy')[0];
GEVdepth_historical_cnt    = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth_historical')[0];

PREC_SCS_Types_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC_SCS_Types')[0];

print("  D4EMLite_PREC_Details Count : " + str(D4EMLite_PREC_Details_cnt));

print("  TEMP2035HotDry Count        : " + str(TEMP2035HotDry_cnt));
print("  TEMP2035Central Count       : " + str(TEMP2035Central_cnt));
print("  TEMP2035WetWarm Count       : " + str(TEMP2035WetWarm_cnt));

print("  TEMP2060HotDry Count        : " + str(TEMP2060HotDry_cnt));
print("  TEMP2060Central Count       : " + str(TEMP2060Central_cnt));
print("  TEMP2060WetWarm Count       : " + str(TEMP2060WetWarm_cnt));

print("  PREC2035HotDry Count        : " + str(PREC2035HotDry_cnt));
print("  PREC2035Central Count       : " + str(PREC2035Central_cnt));
print("  PREC2035WetWarm Count       : " + str(PREC2035WetWarm_cnt));

print("  PREC2060HotDry Count        : " + str(PREC2060HotDry_cnt));
print("  PREC2060Central Count       : " + str(PREC2060Central_cnt));
print("  PREC2060WetWarm Count       : " + str(PREC2060WetWarm_cnt));

print("  GEVdepth2035Stormy Count    : " + str(GEVdepth2035Stormy_cnt));
print("  GEVdepth2035LessStormy Count: " + str(GEVdepth2035LessStormy_cnt));

print("  GEVdepth2060Stormy Count    : " + str(GEVdepth2060Stormy_cnt));
print("  GEVdepth2060LessStormy Count: " + str(GEVdepth2060LessStormy_cnt));

print("  GEVdepth_historical Count   : " + str(GEVdepth_historical_cnt));

print("  PREC_SCS_Types Count        : " + str(PREC_SCS_Types_cnt));
print(" ");


  writing out QA report
  D4EMLite_PREC_Details Count : 5144
  TEMP2035HotDry Count        : 5144
  TEMP2035Central Count       : 5144
  TEMP2035WetWarm Count       : 5144
  TEMP2060HotDry Count        : 5144
  TEMP2060Central Count       : 5144
  TEMP2060WetWarm Count       : 5144
  PREC2035HotDry Count        : 5144
  PREC2035Central Count       : 5144
  PREC2035WetWarm Count       : 5144
  PREC2060HotDry Count        : 5144
  PREC2060Central Count       : 5144
  PREC2060WetWarm Count       : 5144
  GEVdepth2035Stormy Count    : 5144
  GEVdepth2035LessStormy Count: 5144
  GEVdepth2060Stormy Count    : 5144
  GEVdepth2060LessStormy Count: 5144
  GEVdepth_historical Count   : 5144
  PREC_SCS_Types Count        : 5144
 
Wall time: 3.68 s
